# Dual-Input HSN Code Validator (LSTM + CNN)
This notebook implements a dual‐input neural network using TensorFlow/Keras that:

Feeds the HSN code (character‐level) into a bidirectional LSTM to learn valid patterns and catch subtle typos

Processes the product description (word‐level) through a 1D-CNN over pretrained embeddings to extract key semantic cues

Merges both feature streams and performs:

Format validation beyond simple regex rules

Existence checks against our master HSN database

Optional hierarchy enforcement to ensure chapter, heading, and subheading align correctly

In [ ]:
# Necessary Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D, concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
from google.colab import drive #mounting to drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = "/content/drive/MyDrive/HSN_SAC.xlsx" 
sd = pd.read_excel(file_path)
sd.columns = ['HSN_Code', 'Description']
sd.dropna(inplace=True)
sd['HSN_Code'] = sd['HSN_Code'].astype(str)
sd.head() #Reading the first few lines of dataset provided


,HSN_Code,Description
0,01,LIVE ANIMALS
1,0101,"LIVE HORSES, ASSES, MULES AND HINNIES."
2,01011010,"LIVE HORSES, ASSES, MULES AND HINNIES PURE-BRE..."
3,01011020,"LIVE HORSES, ASSES, MULESANDHINNIES PURE-BRED ..."
4,01011090,"LIVE HORSES, ASSES, MULES AND HINNIES PURE-BRE..."


In [ ]:
# Tokenizer for HSN code (character-level)
code_tokenizer = Tokenizer(char_level=True)
code_tokenizer.fit_on_texts(sd['HSN_Code'])
X_code = code_tokenizer.texts_to_sequences(sd['HSN_Code'])
X_code = pad_sequences(X_code, padding='post')

# Tokenizer for description (word-level)
desc_tokenizer = Tokenizer()
desc_tokenizer.fit_on_texts(sd['Description'])
X_desc = desc_tokenizer.texts_to_sequences(sd['Description'])
X_desc = pad_sequences(X_desc, padding='post')

# Encoding HSN codes as class labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(sd['HSN_Code'])

# Splitting the dataset
Xc_train, Xc_test, Xd_train, Xd_test, y_train, y_test = train_test_split(
    X_code, X_desc, y, test_size=0.2, random_state=42
)


In [ ]:
code_input = Input(shape=(X_code.shape[1],), name="hsn_code_input")
code_embed = Embedding(input_dim=len(code_tokenizer.word_index)+1, output_dim=16)(code_input)
code_lstm = LSTM(64)(code_embed)

desc_input = Input(shape=(X_desc.shape[1],), name="description_input")
desc_embed = Embedding(input_dim=len(desc_tokenizer.word_index)+1, output_dim=100)(desc_input)
desc_conv = Conv1D(128, kernel_size=5, activation='relu')(desc_embed)
desc_pool = GlobalMaxPooling1D()(desc_conv)

merged = concatenate([code_lstm, desc_pool])
dense = Dense(64, activation='relu')(merged)
output = Dense(len(np.unique(y)), activation='softmax')(dense)

model = Model(inputs=[code_input, desc_input], outputs=output)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ description_input   │ (None, 313)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hsn_code_input      │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 313, 100)  │    958,900 │ description_inpu… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 8, 16)     │        192 │ hsn_code_input[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 309, 128)  │     64,128 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     20,736 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d[0][0]      │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 192)       │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │     12,352 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 21567)     │  1,401,855 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,458,163 (9.38 MB)

 Trainable params: 2,458,163 (9.38 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    [Xc_train, Xd_train], y_train,
    epochs=10, batch_size=32,
    validation_data=([Xc_test, Xd_test], y_test)
)


Epoch 1/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 50s 88ms/step - accuracy: 0.0000e+00 - loss: 10.0054 - val_accuracy: 0.0000e+00 - val_loss: 10.1993
Epoch 2/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 82s 88ms/step - accuracy: 0.0000e+00 - loss: 9.9238 - val_accuracy: 0.0000e+00 - val_loss: 10.5697
Epoch 3/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 79s 83ms/step - accuracy: 0.0000e+00 - loss: 9.8293 - val_accuracy: 0.0000e+00 - val_loss: 10.9518
Epoch 4/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 48s 88ms/step - accuracy: 6.3308e-04 - loss: 9.5605 - val_accuracy: 0.0000e+00 - val_loss: 11.6598
Epoch 5/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 80s 85ms/step - accuracy: 4.4998e-04 - loss: 9.1249 - val_accuracy: 0.0000e+00 - val_loss: 12.6802
Epoch 6/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 48s 89ms/step - accuracy: 0.0015 - loss: 8.5418 - val_accuracy: 0.0000e+00 - val_loss: 13.7067
Epoch 7/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 81s 88ms/step - accuracy: 0.0047 - loss: 7.9092 - val_accuracy: 0.0000e+00 - val_loss: 15.1338
Epoch 8/10
540/540 ━━━━━━━━━━━━━━━

In [ ]:
def validate_hsn_code(code, description, sd, model, code_tokenizer, desc_tokenizer, label_encoder):
    # Format check
    if not code.isdigit() or not (6 <= len(code) <= 8):
        return {
            "code": code,
            "status": "Invalid",
            "reason": "Invalid format. HSN codes must be 6–8 digits."
        }

    # Preparing the inputs
    seq_code = pad_sequences(code_tokenizer.texts_to_sequences([code]), maxlen=X_code.shape[1], padding='post')
    seq_desc = pad_sequences(desc_tokenizer.texts_to_sequences([description]), maxlen=X_desc.shape[1], padding='post')

    # Predicting
    pred = model.predict([seq_code, seq_desc])
    pred_code = label_encoder.inverse_transform([np.argmax(pred)])[0]
    confidence = float(np.max(pred))

    # Validating against master
    if code in sd['HSN_Code'].values:
        matched_desc = sd.loc[sd['HSN_Code'] == code, 'Description'].values[0]
        return {
            "code": code,
            "status": "Valid",
            "description": matched_desc,
            "confidence": round(confidence, 4)
        }
    else:
        # Checking the hierarchy
        hierarchy = [code[:i] for i in range(len(code)-1, 1, -1)]
        existing = [h for h in hierarchy if h in sd['HSN_Code'].values]
        return {
            "code": code,
            "status": "Invalid",
            "reason": "Code not found in the master dataset.",
            "hierarchy_match": existing,
            "suggested_code": pred_code,
            "confidence": round(confidence, 4)
        }


In [ ]:
# Example of validations
examples = [
    ("01011010", "SWORsdISH (XIPHIAS GLADIUS)"),
    ("03049999", "SHARK"),
    ("ABC123", "INVALID ENTRY"),
    ("03046100", "TILAPIAS (OREOCHROMIS SPP.)"),
    ("12345678", "CATFISH"),
]

for code, desc in examples:
    result = validate_hsn_code(code, desc, sd, model, code_tokenizer, desc_tokenizer, label_encoder)
    print(f"\nHSN Code: {code}")
    for k, v in result.items():
        print(f"  {k}: {v}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

HSN Code: 01011010
  code: 01011010
  status: Valid
  description: LIVE HORSES, ASSES, MULES AND HINNIES PURE-BRED BREEDING ANIMALS HORSES
  confidence: 0.1124
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step

HSN Code: 03049999
  code: 03049999
  status: Invalid
  reason: Code not found in the master dataset.
  hierarchy_match: ['030499', '0304', '03']
  suggested_code: 12119049
  confidence: 0.0154

HSN Code: ABC123
  code: ABC123
  status: Invalid
  reason: Invalid format. HSN codes must be 6–8 digits.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

HSN Code: 03046100
  code: 03046100
  status: Valid
  description: TILAPIAS (OREOCHROMIS SPP.)
  confidence: 0.297
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

HSN Code: 12345678
  code: 12345678
  status: Invalid
  reason: Code not found in the master dataset.
  hierarchy_match: ['12']
  suggested_code: 20079940
  confidence: 0.0183
